### Load data

In [8]:
import sys
import os
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '../')))

from lib.data_preparation import DataPreparation
from lib.preprocessing_data import Preprocessing


data_set = DataPreparation.remove_stopwords(DataPreparation.load_data())
data_set = DataPreparation.clean_data(data_set)


data_set = data_set.sample(frac=0.005).reset_index(drop=True)
data_set

,positive,negative,neutral
0,hey thanks top new followers week much appreci...,back matemticas unhappy,took aol instant messanger money more
1,happy million keep happy million close anoth...,lit unhappy,puritans critics damned
2,kids versus legends crying joy,heart unhappy,set take oath cm today
3,help made sent starters pq open suggestions im...,trying get touch numbers appear down unhappy,shashank manohar quits how
4,thanks me d,damn thats unfortunate unhappy,pill schools
5,congrats winner d eyounji well girl,ohnoo unhappy unhappy hope recuperates sooner,farooq abdullah wins seat total votes


---
# Bag Of Words

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()

import pandas as pd

text_data = data_set.T.fillna('').apply(lambda x: ' '.join(x), axis=1)
vectorizer.fit(text_data)

transformed_output = vectorizer.transform(text_data)
feature_names = vectorizer.get_feature_names_out()
dense_output = transformed_output.todense()
bow_vector = pd.DataFrame(
    dense_output, 
    columns=feature_names,
    index=text_data.index 
)


bow_vector


,abdullah,another,aol,appear,appreciated,back,close,cm,congrats,critics,...,touch,trying,unfortunate,unhappy,versus,votes,week,well,winner,wins
positive,0,1,0,0,1,0,1,0,1,0,...,0,0,0,0,1,0,1,1,1,0
negative,0,0,0,1,0,1,0,0,0,0,...,1,1,1,7,0,0,0,0,0,0
neutral,1,0,1,0,0,0,0,1,0,1,...,0,0,0,0,0,1,0,0,0,1


In [ ]:

word_frequencies = {}
for token_list in tokens:
    for token in token_list:
        word_frequencies[token] = word_frequencies.get(token, 0) + 1


bow_vectors = pd.concat([positive_bow, negative_bow, neutral_bow], axis=0)
bow_vectors.fillna(0, inplace=True)

